In [3]:
from PIL import Image, ImageFont, ImageDraw 
import requests
import io


In [4]:
def get_google_font(font: str, size=24):
    """
    Questa funzione scarica temporaneamente da github.com/google/fonts/ il file .ttf del font desiderato.
    font: stringa indicante il percorso del font su github.com/google/fonts/ nel formato cartella/font_file.ttf
    size: dimensione del font
    """
    url = f"https://github.com/google/fonts/blob/main/ofl/{font}?raw=true"
    print(url)
    r = requests.get(url, allow_redirects=True)
    font = ImageFont.truetype(io.BytesIO(r.content), size=size)

    return font

def crop_image(my_image, aspect_ratio: tuple):
    image  = my_image
    width  = image.size[0]
    height = image.size[1]

    aspect_now = width / float(height)

    ideal_width = aspect_ratio[0]
    ideal_height = aspect_ratio[1]
    ideal_aspect = ideal_width / float(ideal_height)

    if aspect_now > ideal_aspect:
        # Then crop the left and right edges:
        new_width = int(ideal_aspect * height)
        offset = (width - new_width) / 2
        resize = (offset, 0, width - offset, height)
    else:
        # ... crop the top and bottom:
        new_height = int(width / ideal_aspect)
        offset = (height - new_height) / 2
        resize = (0, offset, width, height - offset)

    image = image.crop(resize).resize((ideal_width, ideal_height), Image.LANCZOS)
    # img_path_cropped = "CROPPED_"+img_path
    # thumb.save(img_path_cropped)
    return image

def write_on_image(my_image, edit_dict):
    image_editable = ImageDraw.Draw(my_image)

    for scope in edit_dict.keys():
        # Get textbox dimensions from coordinates
        box_w = edit_dict[scope]["textbox"][1][0] - edit_dict[scope]["textbox"][0][0]
        box_h = edit_dict[scope]["textbox"][1][1] - edit_dict[scope]["textbox"][0][1]

        # Get appropriate font size to make the text fit the box in a single line
        max_size = edit_dict[scope]["size"] # Start from default font size
        text_length = get_google_font(edit_dict[scope]["font"], max_size).getlength(edit_dict[scope]["text"]) # Get text length given font and size
        # Finché il testo non ci sta nel box predefinito, riduco la size di 2 punti e riprovo
        while text_length > box_w:
            max_size -= 2
            text_length = get_google_font(edit_dict[scope]["font"], max_size).getlength(edit_dict[scope]["text"])

        image_editable.text(edit_dict[scope]["position"], 
                            edit_dict[scope]["text"],
                            edit_dict[scope]["color"],
                            anchor=edit_dict[scope]["anchor"],
                            font=get_google_font(edit_dict[scope]["font"]
                                                    , max_size))
    
    return my_image

def image_edit(img_path: str, edit_dict: dict, crop=True):
    
    if crop:
        # Carico immagine
        my_image = Image.open(img_path) 
        # Crop immagine
        my_image = crop_image(my_image, (648,1152))
    else:
        # Carico immagine
        my_image = Image.open(img_path) 

    my_image = write_on_image(my_image, edit_dict)

    # Salvataggio risutlato
    my_image.save("EDIT_"+img_path)

In [5]:
img_path = "test_img.png"

# da fare:
# - mettere posizione parametrica (e.g. titolo centrato)
# https://stackoverflow.com/questions/1970807/center-middle-align-text-with-pil
# https://pillow.readthedocs.io/en/stable/handbook/text-anchors.html
# - scegliere colore testo parametrico, adatto all'immagine


edit_dict = {
    "title": {"text": "Torneo di Bang",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 90, 
                "color": (55, 0, 134),
                "position":(324,100),
                "anchor":"mm"},
    "body": {"text": "",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 80, 
                "color": (55, 0, 134),
                "position":(0,0),
                "anchor":"mm"},
    "location": {"text": "Centro NOI di Povegliano",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,900),
                "anchor":"lb"},
    "time": {"text": "Mercoledì 31 febbraio ore 21:00",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,950),
                "anchor":"lb"},
    "time": {"text": "21:00",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,950),
                "anchor":"lb"},
    "cost": {"text": "Iscrizione libera",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,1000),
                "anchor":"lb"}
}

img_path_edit = image_edit(img_path, edit_dict)

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [9]:
img = Image.open("test_img.png")
draw = ImageDraw.Draw(img)
font = get_google_font("montserratalternates/MontserratAlternates-Bold.ttf"
                                                    , 80)

# width,height= draw.textsize("Sample Text", font=font)
draw.multiline_text((100, 100),"Sample Text Sample Text Sample Text Sample Text",(255,255,255),font=font, align='center')
img.save('sample-out.jpg')

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [13]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

astr = '''The rain in Spain falls mainly on the plains.'''
para = textwrap.wrap(astr, width=15)

MAX_W, MAX_H = 648,1152
img = Image.open("test_img.png")
img = crop_image(img, (648,1152))
draw = ImageDraw.Draw(img)
font = get_google_font("montserratalternates/MontserratAlternates-Bold.ttf"
                                                    , 80)

current_h, pad = 50, 10
for line in para:
    w, h = draw.textlength(line, font=font), 80
    draw.text(((MAX_W - w) / 2, current_h), line, font=font)
    current_h += h + pad

img.save('test.png')

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [1]:
int(1.5)

1

In [22]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
img = Image.open("test_img.png")
img = crop_image(img, (648,1152))

draw = ImageDraw.Draw(img)
draw.rectangle(((50, 30), (600, 170)), width=3) # TITOLO
draw.rectangle(((50, 170), (600, 270)), width=3) # BODY
draw.rectangle(((50, 860), (324, 920)), width=3) # DATE
draw.rectangle(((50, 920), (324, 980)), width=3) # TIME
draw.rectangle(((50, 980), (324, 1040)), width=3) # LOCATION
draw.rectangle(((50, 1040), (324, 1100)), width=3) # COST



img.save('layout_example.png')

In [2]:
import sys
from pathlib import Path
sys.path.insert(0,str(Path.cwd().parent))
import utils.aws_utils as aws_utils

In [4]:
sys.path.insert(0,'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA')

In [5]:
sys.path

['c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\lambda_functions',
 'C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv',
 '',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Lib\\site-packages',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Lib\\site-packages\\win32',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Lib\\site-packages\\Pythonwin',
 'c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\lambda_functions']

In [1]:
import os

In [3]:
print(os.environ["PATH"])

c:\Users\rican\OneDrive\Documenti\GitHub\LudusSMA\.venv\Scripts;C:\Users\rican\OneDrive\Documenti\GitHub\LudusSMA\.venv\Scripts;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Amazon\AWSCLIV2\;C:\Program Files\Git\cmd;C:\Users\rican\AppData\Local\Programs\Python\Python311\Scripts\;C:\Users\rican\AppData\Local\Programs\Python\Python311\;C:\Users\rican\AppData\Local\Microsoft\WindowsApps;C:\Users\rican\AppData\Local\GitHubDesktop\bin;;C:\Users\rican\AppData\Local\Programs\Microsoft VS Code\bin


In [5]:
os.environ["PATH"]+='c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA'


In [9]:
os.environ["PATH"]=os.environ["PATH"].replace('c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA', ';c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA')

In [10]:
os.environ["PATH"]

';c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Scripts;C:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA\\.venv\\Scripts;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\Program Files\\Amazon\\AWSCLIV2\\;C:\\Program Files\\Git\\cmd;C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311\\Scripts\\;C:\\Users\\rican\\AppData\\Local\\Programs\\Python\\Python311\\;C:\\Users\\rican\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\rican\\AppData\\Local\\GitHubDesktop\\bin;;C:\\Users\\rican\\AppData\\Local\\Programs\\Microsoft VS Code\\bin;c:\\Users\\rican\\OneDrive\\Documenti\\GitHub\\LudusSMA'

# TEST SEZIONE CONTATTI

In [7]:

import requests
import io

def crop_image(my_image, aspect_ratio: tuple):
    image  = my_image
    width  = image.size[0]
    height = image.size[1]

    aspect_now = width / float(height)

    ideal_width = aspect_ratio[0]
    ideal_height = aspect_ratio[1]
    ideal_aspect = ideal_width / float(ideal_height)

    if aspect_now > ideal_aspect:
        # Then crop the left and right edges:
        new_width = int(ideal_aspect * height)
        offset = (width - new_width) / 2
        resize = (offset, 0, width - offset, height)
    else:
        # ... crop the top and bottom:
        new_height = int(width / ideal_aspect)
        offset = (height - new_height) / 2
        resize = (0, offset, width, height - offset)

    image = image.crop(resize).resize((ideal_width, ideal_height), Image.LANCZOS)
    # img_path_cropped = "CROPPED_"+img_path
    # thumb.save(img_path_cropped)
    return image
def get_google_font(font: str, size=24):
    # MIGLIORIA: SCARICARE IL FONT SOLO UNA VOLTA E TENERLO BUONO PER TUTTE LE SEZIONI
    # E' POSSIBILE? FORSE NO PERCHé VA SCARICATO PER LA SIZE SPECIFICA DI QUEL TESTO
    """
    Questa funzione scarica temporaneamente da github.com/google/fonts/ il file .ttf del font desiderato.
    font: stringa indicante il percorso del font su github.com/google/fonts/ nel formato cartella/font_file.ttf
    size: dimensione del font
    """
    url = f"https://github.com/google/fonts/blob/main/ofl/{font}?raw=true"
    r = requests.get(url, allow_redirects=True)
    font = ImageFont.truetype(io.BytesIO(r.content), size=size)

    return font

In [19]:
# TEST SEZIONE CONTATTI
ICONS_PATH = "../../images/icons/"
# contacts = {
#     "email": {"text": "ludusgate@gmail.com",
#                 "icon":"email-color.png"},
#     "whatsapp": {"text": "34X XXX XX XX",
#                 "icon":"whatsapp-logo-color.png"},
#     "insta_dm": {"text": "DM: @ludusgate",
#                 "icon":"instagram-color.png"}
# }
# contacts_font_size = 30
# color = (255,255,255)
# STROKE_WIDTH = 2
# STROKE_COLOR = (0,0,0)
# font = "montserratalternates/MontserratAlternates-Bold.ttf"
# contacts_start_point = (50,1000)
# contacts_end_point = ()

from PIL import Image, ImageDraw, ImageFont
img = Image.open("test_img.png")
img = crop_image(img, (648,1152))

image_editable = ImageDraw.Draw(img)


writing_cursor_y = contacts_start_point[1]
for contact in contacts.keys():
    writing_cursor_x = contacts_start_point[0]
    print(contact)
    # Get icon
    icon_path = ICONS_PATH+contacts[contact]["icon"]
    icon = Image.open(icon_path)
    # Resize icon
    icon.thumbnail((contacts_font_size, contacts_font_size), Image.Resampling.LANCZOS)
    # Paste the icon on the image
    icon_position = (int(writing_cursor_x), int(writing_cursor_y))
    img.paste(icon, icon_position, mask=icon)
    writing_cursor_x += contacts_font_size+5 # più un margine

    text = contacts[contact]["text"]
    text_position = (int(writing_cursor_x), int(writing_cursor_y))
    # text_len = get_google_font(font, contacts_font_size).getlength(text)

    image_editable.text(text_position, text,
                        color,
                        font=get_google_font(font, contacts_font_size),
                                    stroke_width=STROKE_WIDTH,
                                    stroke_fill=STROKE_COLOR)
    writing_cursor_y += contacts_font_size+5




img.save("TEST_CONTACTS.png")



email
whatsapp
insta_dm


In [20]:
import sys
sys.path

['c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\lambda_functions\\create_ig_stories',
 'C:\\Users\\Riccardo\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\Riccardo\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\Riccardo\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\Riccardo\\AppData\\Local\\Programs\\Python\\Python311',
 'c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\venv',
 '',
 'c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\venv\\Lib\\site-packages',
 'c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\venv\\Lib\\site-packages\\win32',
 'c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\venv\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\Riccardo\\Documents\\GitHub\\LudusSMA\\venv\\Lib\\site-packages\\Pythonwin']